In [1]:
# !pip install yfinance
# !pip install prophet
# !pip install matplotlib

In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

c:\Users\fafaf\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
stock_symbols = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'meta', 'JPM', 'JNJ', 'V',
    'PG', 'DIS', 'MA', 'UNH', 'HD', 'PYPL', 'BAC', 'NFLX', 'XOM', 'VZ',
    'KO', 'PEP', 'MRK', 'INTC', 'T', 'PFE', 'CSCO', 'WMT', 'BA', 'NKE',
    'GS', 'MCD', 'ADBE', 'COST', 'IBM', 'CRM', 'ORCL', 'WFC', 'MDT', 'C',
    # Add more symbols to reach 500
]

In [4]:
# Fetch historical data for a single stock from Yahoo Finance
def fetch_stock_data(symbol, start_date='2015-01-01'):
    data = yf.download(symbol, start=start_date)
    data.reset_index(inplace=True)
    # Prophet requires columns as 'ds' and 'y'
    stock_data = data[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
    return stock_data

In [5]:
# Configure and train Prophet model for each stock
def train_prophet(stock_data, stock_name, yearly_seasonality=True, daily_seasonality=False, holidays=None, changepoint_prior_scale=0.05):
    # Initialize Prophet with customized parameters
    m = Prophet(yearly_seasonality=yearly_seasonality,
                    daily_seasonality=daily_seasonality,
                    holidays=holidays,
                    changepoint_prior_scale=changepoint_prior_scale)

    # Fit the model to stock data
    m.fit(stock_data)

    # Forecast for the next 365 days
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)

    # Plot forecast
    fig = m.plot(forecast)
    plt.title(f'Forecast for {stock_name}')
    plt.show()

    # Return the forecast for further analysis
    return forecast

In [6]:
# Function to iterate over all stocks and forecast their prices
def forecast_stocks(symbols, start_date='2015-01-01'):
    for symbol in symbols:
        print(f"Processing stock: {symbol}")
        # Fetch stock data
        stock_data = fetch_stock_data(symbol, start_date)

        # Train Prophet model
        forecast = train_prophet(stock_data, symbol)

        # Save the forecast as CSV
        forecast.to_csv(f'forecast_{symbol}.csv', index=False)

In [7]:
# Customize Prophet parameters (you can tweak them in the notebook)
yearly_seasonality = True
daily_seasonality = False
holidays = None  # You can define holidays if needed
changepoint_prior_scale = 0.05  # Control the flexibility of the model